In [ ]:
#Arquivos em PNG estao sem o metadados e mais avermelhados

In [ ]:
# =========================================================
# 1) Instalar dependências
# =========================================================
!apt-get update -y
!apt-get install -y exiftool imagemagick
!pip install pillow wand gradio

from wand.image import Image as WandImage
from PIL import Image
import os
import shutil
import gradio as gr
import time

# Instalar Ghostscript (necessário para AI/EPS/PDF)
!apt-get install -y ghostscript

# Liberar conversão de PDF/AI/EPS no ImageMagick
policy_file="/etc/ImageMagick-6/policy.xml"

# Faz backup (por segurança)
!cp $policy_file ${policy_file}.backup

# Remove linhas que bloqueiam PDF, PS, EPS
!sed -i '/PDF/d' $policy_file
!sed -i '/PS/d' $policy_file
!sed -i '/EPS/d' $policy_file

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,820 kB]
Get:14 http://archiv

In [ ]:
# =========================================================
# 2) Funções de conversão (com camadas mescladas)
# =========================================================
def convert_with_wand(input_path, output_path):
    with WandImage(filename=input_path) as img:
        img = img.clone()
        img.merge_layers('flatten')
        img.format = output_path.split(".")[-1].upper()
        img.save(filename=output_path)

def convert_with_pillow(input_path, output_path):
    img = Image.open(input_path)
    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")
    img.save(output_path, quality=95)


# =========================================================
# 3) Função principal
# =========================================================
def process_files(file_list, formato):
    if not file_list:
        return None

    formato = formato.lower()
    output_folder = f"convertidas_{int(time.time())}"
    os.makedirs(output_folder, exist_ok=True)

    outputs = []

    for file in file_list:
        filename = os.path.basename(file.name)
        base, ext = os.path.splitext(filename)
        ext = ext.lower()

        src_path = file.name
        output_path = os.path.join(output_folder, f"{base}.{formato}")

        try:
            if ext in [".psd", ".ai", ".eps", ".svg", ".pdf"]:
                convert_with_wand(src_path, output_path)
            else:
                convert_with_pillow(src_path, output_path)

            os.system(f'exiftool -TagsFromFile "{src_path}" -all:all "{output_path}" -overwrite_original')
            outputs.append(output_path)

        except Exception as e:
            print(f"Erro com {filename}: {e}")

    # ✅ Se apenas um arquivo → baixar diretamente
    if len(outputs) == 1:
        return outputs[0]

    # ✅ Se mais de um arquivo → zipar
    zip_name = output_folder + ".zip"
    shutil.make_archive(output_folder, "zip", output_folder)
    return zip_name


# =========================================================
# 4) Interface (Gradio) — simples e direta
# =========================================================
ui = gr.Interface(
    fn=process_files,
    inputs=[
        gr.Files(label="Arraste ou selecione as imagens aqui"),
        gr.Dropdown(["jpg", "png"], label="Formato de saída", value="jpg")
    ],
    outputs=gr.File(label="Download"),
    title="Conversor de Imagens (camadas mescladas + metadados preservados)",
    allow_flagging="never"
)

ui.launch(share=True, debug=False)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d26816dacb59358385.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
